<a href="https://colab.research.google.com/github/jantic/DeOldify/blob/master/DeOldify-video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ <font color='#FF000'>[ </font> DeOldify-video

##This Colab notebook colorizes video in four steps
1. Upload source media or specify media URL - YouTube, Twitter, MySpace, etc.
2. Extract single images from media
3. Process images with [DeOldify](https://github.com/jantic/DeOldify) 
4. Rebuild the video from **<font color='#CC0000'>c</font><font color='#CC8800'>o</font><font color='#FFBB00'>l</font><font color='#00DD00'>o</font><font color='#0000FF'>r</font>ized** images

I'm on twitter [@tradica](https://twitter.com/tradica)


---


Thanks [@citnaj](https://twitter.com/citnaj) for creating DeOldify and thanks to Matt Robinson for his [notebook](https://colab.research.google.com/github/jantic/DeOldify/blob/master/DeOldify_colab.ipynb). It helped make DeOldify approachable.





---


#◢ <font color='#FF000'>[ </font> Set Runtime type to Python 3/GPU
In the Runtime menu above be sure:
* Runtime Type = Python 3
* Hardware Accelerator = GPU **<font color='#FF000'><-------------- IMPORTANT </font>**


In [ ]:
from os import path
import torch
print(torch.__version__)
print(torch.cuda.is_available())

#◢ <font color='#FF000'>[ </font> Git clone and install DeOldify

In [ ]:
!git clone -b FastAIv1 --single-branch https://github.com/jantic/DeOldify.git DeOldify
#!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
cd DeOldify

#◢ <font color='#FF000'>[ </font> Setup

In [ ]:
!pip install PyDrive
!pip install ffmpeg-python
!pip install youtube-dl
!pip install tensorboardX

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
from IPython.display import Image
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.tensorboard import *
from fasterai.loss import *
from fasterai.filters import *
from fasterai.generators import *
from pathlib import Path
from itertools import repeat
from google.colab import drive
from google.colab import files
torch.backends.cudnn.benchmark=True

In [ ]:
!mkdir "models"

In [ ]:
!wget https://www.dropbox.com/s/3e4dqky91h824ik/colorize_gen.pth -O ./models/colorize_gen.pth

In [ ]:
!mkdir /content/WORKFOLDER
!mkdir /content/WORKFOLDER/monochromatics
!mkdir /content/WORKFOLDER/colorized
!mkdir /content/WORKFOLDER/ANSWER

import ffmpeg

# COLAB PROGRESS BAR
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 40%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
results_dir=Path('/content/WORKFOLDER/colorized')

In [ ]:
vis = get_colorize_visualizer(results_dir=results_dir)

In [ ]:
%cd "/content/WORKFOLDER"
media_file = "/content/WORKFOLDER/media_file"

def ugly_get_fps():
  media_data = ffmpeg.probe(media_file)
  stream_data = media_data['streams']
  stream_data_zero = str(stream_data).rsplit('avg_frame_rate\': \'', 1)[1]
  fps_nums=stream_data_zero.rsplit("'")[0]
  fps_list = fps_nums.rsplit("/")
  return int(fps_list[0])/int(fps_list[1])

---
#◢ <font color='#FF000'>[ </font>1<font color='#FF000'> ] </font> Upload Source Video or Specify Media URL

###◢  <font color='#FF000'>[ </font> Upload

In [ ]:
#Run this to make an upload widget appear
source_media = files.upload()

os.system('ln -s /content/WORKFOLDER/' + list(source_media.keys())[0] + ' /content/WORKFOLDER/media_file')
source_media_name = list(source_media.keys())[0]

###◢  <font color='#FF000'>[ </font> Specify URL
YouTube, Imgur, Twitter, MySpace, Reddit ... most work

In [ ]:
#@title Paste media URL or leave blank if uploading
media_url = '' #@param {type:"string"}
source_media_name = media_url.rsplit('/', 1)[-1]
os.system('youtube-dl ' + media_url + ' -o ' + source_media_name)
os.system('ln -s /content/WORKFOLDER/' + str(source_media_name) + ' /content/WORKFOLDER/media_file')

#◢ <font color='#FF000'>[ </font>2<font color='#FF000'> ] </font> Extract Images from Source Media

In [ ]:
os.system('ffmpeg -i media_file -qscale:v 2 /content/WORKFOLDER/monochromatics/%5d.jpg')
framecount = len(os.listdir('/content/WORKFOLDER/monochromatics'))

#◢ <font color='#FF000'>[ </font>3<font color='#FF000'> ] </font> DeOldify / Colorize

In [ ]:
render_factor = 21  #@param {type: "slider", min: 5, max: 42}
#progress bar
prog = 0
out = display(progress(0, 100), display_id=True)
for img in os.listdir("/content/WORKFOLDER/monochromatics"):
  img_path = str("/content/WORKFOLDER/monochromatics/") + img
  if os.path.isfile(img_path):
    qqq = vis.get_transformed_image(img_path, render_factor)
    qqq.save("/content/WORKFOLDER/colorized/" + img)
  prog += 1
  out.update(progress(prog, framecount))

#◢ <font color='#FF000'>[ </font>4<font color='#FF000'> ] </font> Build Video

In [ ]:
answer_file = '/content/WORKFOLDER/ANSWER/' + source_media_name + '-RF' + str(render_factor) + 'FR' + str(round(ugly_get_fps(), 2)) + '.mp4'
os.system('ffmpeg -f image2 -framerate ' + str(ugly_get_fps()) + ' -i /content/WORKFOLDER/colorized/%05d.jpg -c:v libx264 -crf 0 ' + answer_file)

###◢  <font color='#FF000'>[ </font> Download
* In the Menu on the left, click **Files**
* It's in /content/WORKFOLDER/ANSWER/
* ( Or use this with its zero feedback :) )

In [ ]:
#Find your video in Files >  /content/WORKFOLDER/ANSWER/  (It's better than this)
files.download(answer_file)

###◢  <font color='#FF000'>[ </font> Build 50/50 Split Screen original / DeOldified video
(need to complete step four)

In [ ]:
os.system('ffmpeg -i ' + str(media_file) + ' -i ' + str(answer_file) + ' -filter_complex  "[0]crop=iw/2:ih:0:0,  pad=iw*2:ih[left];  [1]crop=iw/2:ih:iw/2:0[right]; [left][right]overlay=w" -strict -2 /content/WORKFOLDER/ANSWER/split-RF' + str(render_factor) + '.mp4')

---
#⚙ <font color='#FF000'>[ </font> Delete Workfiles

In [ ]:
# Delete extracted
!rm /content/WORKFOLDER/monochromatics/*jpg

In [ ]:
# Delete Colorized
!rm /content/WORKFOLDER/colorized/*jpg

In [ ]:
!rm /content/WORKFOLDER/media_file

In [21]:
# Delete Source Media 
os.system('rm /content/WORKFOLDER/' + source_media_name)

0

In [22]:
# Delete Answer
os.system('rm ' + answer_file)

0

---
#⚙ <font color='#FF000'>[ </font>  
* [/r/Nickelodeons/](https://www.reddit.com/r/Nickelodeons/)
* https://twitter.com/silentmoviegifs 

https://twitter.com/silentmoviegifs/status/1087282910288363522
If you click the date at the top of a tweet you can copy the URL from the browser